In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
from gurobipy import GRB

In [2]:
# df = pd.read_excel('activity_mandatory.xlsx') #要改为全部的
# df = pd.read_excel('D:\\course schedule\\course2301\\activity_mandatory_all.xlsx') 
# df = pd.read_excel('activity_mandatory_all.xlsx') 

# 用于获取所有需要排的课程信息并储存在列表以及字典中（包括Mandatory,Optional（非任何Mandatory的）,Special（被人为处理的Special））
df_whole_data = pd.read_excel('POS-Activity总表.xlsx', sheet_name="Sheet1")
df_whole = df_whole_data[['Department', 'sub_POS', 'Type', 'Module Code', 'Activity Counts', 'Normalized Weight', 'Duration', 'Count']]

Pos_ModuleCode = df_whole.groupby('sub_POS')['Module Code'].apply(set).to_dict() 
ModuleCode_Pos = df_whole.groupby('Module Code')['sub_POS'].apply(set).to_dict()
# print(ModuleCode_Pos)

df_ModuleCode_Pos = pd.DataFrame(list(ModuleCode_Pos.items()), columns=['sub_POS', 'Module Codes'])

# 将集合转换为字符串，以便在Excel中显示
df_ModuleCode_Pos['Module Codes'] = df_ModuleCode_Pos['Module Codes'].apply(lambda x: ', '.join(x))

# 输出到Excel文件
df_ModuleCode_Pos.to_excel('test.xlsx', index=False)

Courses, x = gp.multidict(ModuleCode_Pos)
Pos = list(Pos_ModuleCode.keys()) #a list of all major
Timeslots = range(8)
print(len(Courses))
print(len(Pos))

682
1491


In [3]:
# 以下列表+字典均：全部的Mandatory+人工Special+纯Optional

Courses

# ModuleCode_Pos：以活动名为键，专业为值的字典
# ModuleCode_Pos

# Pos_ModuleCode： 以专业为键，活动名为值的字典
# Pos_ModuleCode

['ACT2111', 'ACT3011_01', 'ACT3011_02', 'ACT3011_03', 'ACT3121_01', 'ACT3121_02', 'ACT3121_03', 'ACT3131_01', 'ACT3131_02', 'ACT3141_01', 'ACT3141_02', 'ACT3161_01', 'ACT3161_02', 'ACT3161_03', 'ACT3321', 'ACT4111', 'ACT4213_01', 'ACT4213_02', 'ACT4214', 'ACT4253_01', 'ACT4253_02', 'ACT4253_03', 'ACT4253_04', 'ACT4311', 'BIM2006', 'BIM3001', 'BIM3009', 'BIM3011', 'BIM3019', 'BIO1001', 'BIO2004', 'BIO2104', 'BIO3201', 'BIO3204', 'BIO3207', 'BIO3213', 'BIO3301', 'BIO4102', 'BIO4201', 'BIO4202', 'BIO4204', 'BIO4902', 'BME2001', 'BME3006', 'BME3200', 'BME4001', 'BME4003', 'BME4007', 'BME4009', 'BME4010', 'CHI1000_01', 'CHI1000_02', 'CHI1000_03', 'CHI1000_04', 'CHI1000_05', 'CHI1000_06', 'CHI1000_07', 'CHI1000_08', 'CHI1000_09', 'CHI1000_10', 'CHI1000_11', 'CHI1000_12', 'CHI1000_13', 'CHI1000_14', 'CHI2004', 'CHI2011', 'CHI4001', 'CHI4002', 'CHI4006', 'CHM2004', 'CHM2118', 'CHM2320', 'CHM3130', 'CHM3210', 'CHM3418', 'CHM3520', 'CHM3540', 'CHM4328', 'CHM4422', 'CLC1101_01', 'CLC1101_02', 'CL

In [4]:
count = 0
for key, value in Pos_ModuleCode.items():
    if count < len(Pos):
        print(f"{key}: {value}")
        count += 1
    else:
        break

ACT_ADA-Year02: {'ECO2011_02', 'ECO2121_04', 'ECO3211', 'FIN2010_05', 'FIN2010_03', 'FIN2010_04', 'ECO3021_02', 'MGT2020_01', 'MGT2020_08', 'ECO3410', 'MGT3010_01', 'ECO3121_03', 'FIN2010_07', 'FIN2010_08', 'MGT2020_04', 'ECO3011_02', 'ECO2121_05', 'ECO2121_07', 'ECO3121_02', 'MGT3010_02', 'ECO2121_03', 'FIN2020_01', 'ACT2111', 'MGT2020_06', 'ACT3161_03', 'MGT2020_03', 'MGT2020_07', 'ECO2121_01', 'ACT3161_01', 'FIN2010_01', 'ECO2121_02', 'MGT2020_02', 'MIS2051_04', 'ECO2011_01', 'ECO3021_01', 'FIN2010_06', 'MIS2051_02', 'ECO2121_06', 'FIN2020_03', 'ECO3011_01', 'FIN2020_02', 'FIN2010_02', 'ACT3321', 'MIS2051_03', 'MIS2051_01', 'MGT2020_05', 'ECO3121_01', 'ACT3161_02'}
ACT_ADA-Year02_01: {'DMS2030_01', 'DMS2030_04', 'ENG2002B_01', 'ACT3131_02', 'DMS2030_05', 'ENG2002B_20', 'ACT3131_01', 'ACT3121_03', 'ENG2002B_18', 'DMS2030_03', 'ENG2002B_02', 'ENG2002B_14', 'ENG2002B_17', 'ENG2002B_25', 'ACT3011_01', 'ENG2002B_10', 'ENG2002B_16', 'ENG2002B_31', 'ACT3141_02', 'ENG2002B_06', 'ENG2002B_08

In [5]:
#构建模型
course_optimize = gp.Model("Optimization Model")


n = len(Pos) # major数量
m = len(Courses)  # activities数量
t = 8   # 可供选择的timeslot

# 创建三维变量 X 
X = course_optimize.addVars(n, m, t, vtype=GRB.BINARY, name="X")
# y = course_optimize.addVars(8, m, vtype=GRB.BINARY, name="y") #A binary variable indicating whether a course is assigned to a specific time slot
# print("n,m,t = ",n,m,t)
print("major n:", n)
print("activities m:", m)
print("timeslot t:", t)

major n: 1491
activities m: 682
timeslot t: 8


In [6]:
# 用于获取所有Mandatory的课程信息并储存在列表以及字典中

df_Mandatory = df_whole[(df_whole['Type'] == 'Mandatory')].copy()

Pos_ModuleCode_Mandatory = df_Mandatory.groupby('sub_POS')['Module Code'].apply(set).to_dict()
ModuleCode_Pos_Mandatory = df_Mandatory.groupby('Module Code')['sub_POS'].apply(set).to_dict()

# filtered_Courses包含所有Mandatory
filtered_Courses, z = gp.multidict(ModuleCode_Pos_Mandatory)

# 构建3个字典，分别用于：专业找坐标；课程找坐标；坐标找课程
pos_to_index = {pos: index for index, pos in enumerate(Pos)} # i坐标
index_to_pos = {index: pos for pos, index in pos_to_index.items()}

course_to_index = {course: index for index, course in enumerate(Courses)}
index_to_course = {index: course for course, index in course_to_index.items()}

In [7]:
index_to_course

{0: 'ACT2111',
 1: 'ACT3011_01',
 2: 'ACT3011_02',
 3: 'ACT3011_03',
 4: 'ACT3121_01',
 5: 'ACT3121_02',
 6: 'ACT3121_03',
 7: 'ACT3131_01',
 8: 'ACT3131_02',
 9: 'ACT3141_01',
 10: 'ACT3141_02',
 11: 'ACT3161_01',
 12: 'ACT3161_02',
 13: 'ACT3161_03',
 14: 'ACT3321',
 15: 'ACT4111',
 16: 'ACT4213_01',
 17: 'ACT4213_02',
 18: 'ACT4214',
 19: 'ACT4253_01',
 20: 'ACT4253_02',
 21: 'ACT4253_03',
 22: 'ACT4253_04',
 23: 'ACT4311',
 24: 'BIM2006',
 25: 'BIM3001',
 26: 'BIM3009',
 27: 'BIM3011',
 28: 'BIM3019',
 29: 'BIO1001',
 30: 'BIO2004',
 31: 'BIO2104',
 32: 'BIO3201',
 33: 'BIO3204',
 34: 'BIO3207',
 35: 'BIO3213',
 36: 'BIO3301',
 37: 'BIO4102',
 38: 'BIO4201',
 39: 'BIO4202',
 40: 'BIO4204',
 41: 'BIO4902',
 42: 'BME2001',
 43: 'BME3006',
 44: 'BME3200',
 45: 'BME4001',
 46: 'BME4003',
 47: 'BME4007',
 48: 'BME4009',
 49: 'BME4010',
 50: 'CHI1000_01',
 51: 'CHI1000_02',
 52: 'CHI1000_03',
 53: 'CHI1000_04',
 54: 'CHI1000_05',
 55: 'CHI1000_06',
 56: 'CHI1000_07',
 57: 'CHI1000_08',
 

In [8]:
# 约束条件1

# 集合Mandatory用于储存每个专业的课程（对应的坐标）
Mandatory = {}
for pos in Pos:
    i = pos_to_index[pos]
    course_i = []
    for course in filtered_Courses:
        if pos in ModuleCode_Pos_Mandatory.get(course, []):
            course_i.append(course_to_index[course])
    Mandatory[i] = course_i


for i in range(n):  # 遍历每个专业
    mani = Mandatory[i]
    print(f"专业 {Pos[i]} 可以选择的课程：")
    for j in mani:
        print(index_to_course[j], end=' ')
    print('\n')

# 添加约束条件
for i in range(n):
    mani = Mandatory[i]
    for k in range(t):
        course_optimize.addConstr(gp.quicksum(X[i, j, k] for j in mani) <= 1)

#约束条件2
"""for i in range(n):
    for j in range(m):
        course_optimize.addConstr(gp.quicksum(X[i, j, k] for k in range(t)) <= 1)"""

#约束条件3-1
# data_num_mandatory = pd.read_excel('#of mandatory course of POSi.xlsx')
# slack_vars = []

# for i in range(n):
#     for j in range(m):
#         slack_var = course_optimize.addVar(lb=0, name=f"slack_{Pos[i]}_{Courses[j]}")
#         slack_vars.append(slack_var)
#         course_optimize.addConstr(gp.quicksum(X[i, j, k] for k in range(t)) + slack_var == data_num_mandatory.iloc[i]['Number of Mandatory Courses'])

#约束条件3-2
    # for k in range(t):
    #     course_optimize.addConstr(gp.quicksum(X[i, j, k] for j in range(m)) == 1)











专业 ACT_ADA-Year02 可以选择的课程：


专业 ACT_ADA-Year02_01 可以选择的课程：
ACT3011_01 ACT3011_02 ACT3011_03 ACT3121_01 ACT3121_02 ACT3121_03 ACT3131_01 ACT3131_02 ACT3141_01 ACT3141_02 DMS2030_01 DMS2030_02 DMS2030_03 DMS2030_04 DMS2030_05 ENG2002B_01 ENG2002B_02 ENG2002B_03 ENG2002B_04 ENG2002B_05 ENG2002B_06 ENG2002B_07 ENG2002B_08 ENG2002B_09 ENG2002B_10 ENG2002B_11 ENG2002B_12 ENG2002B_13 ENG2002B_14 ENG2002B_15 ENG2002B_16 ENG2002B_17 ENG2002B_18 ENG2002B_19 ENG2002B_20 ENG2002B_21 ENG2002B_22 ENG2002B_23 ENG2002B_24 ENG2002B_25 ENG2002B_26 ENG2002B_27 ENG2002B_28 ENG2002B_29 ENG2002B_30 ENG2002B_31 ENG2002B_32 

专业 ACT_ADA-Year02_02 可以选择的课程：
ACT3121_01 ACT3121_02 ACT3121_03 ACT3131_01 ACT3131_02 ACT3141_01 ACT3141_02 DMS2030_01 DMS2030_02 DMS2030_03 DMS2030_04 DMS2030_05 ENG2002B_01 ENG2002B_02 ENG2002B_03 ENG2002B_04 ENG2002B_05 ENG2002B_06 ENG2002B_07 ENG2002B_08 ENG2002B_09 ENG2002B_10 ENG2002B_11 ENG2002B_12 ENG2002B_13 ENG2002B_14 ENG2002B_15 ENG2002B_16 ENG2002B_17 ENG2002B_18 ENG2002B_19 

'for i in range(n):\n    for j in range(m):\n        course_optimize.addConstr(gp.quicksum(X[i, j, k] for k in range(t)) <= 1)'

In [9]:
# 读取Module Code & Activity Counts

df_code_count = pd.read_excel('全部非TUT Course(from teample).xlsx') # df_code_count包含了所有课程

# 构建字典用于：课程找权重
ModuleCode_Weight_Mandatory = df_Mandatory.groupby('Module Code')['Normalized Weight'].apply(set).to_dict()
Weights, x = gp.multidict(ModuleCode_Weight_Mandatory)

"""course_to_index_whole = {course: index for index, course in enumerate(Courses)}
Pos_to_index_whole = {pos: index for index, pos in enumerate(Pos)}"""


'course_to_index_whole = {course: index for index, course in enumerate(Courses)}\nPos_to_index_whole = {pos: index for index, pos in enumerate(Pos)}'

In [10]:
ModuleCode_Weight_Mandatory

{'ACT2111': {0.07261653465346535},
 'ACT3011_01': {0.06380907590759077},
 'ACT3011_02': {0.06380907590759077},
 'ACT3011_03': {0.06380907590759077},
 'ACT3121_01': {0.06380907590759077},
 'ACT3121_02': {0.06380907590759077},
 'ACT3121_03': {0.06380907590759077},
 'ACT3131_01': {0.04950945544554456},
 'ACT3131_02': {0.04950945544554456},
 'ACT3141_01': {0.04950945544554456},
 'ACT3141_02': {0.04950945544554456},
 'ACT3161_01': {0.06380907590759077},
 'ACT3161_02': {0.06380907590759077},
 'ACT3161_03': {0.06380907590759077},
 'ACT3321': {0.04621415841584159},
 'ACT4111': {0.07261653465346535},
 'ACT4213_01': {0.06931123762376239},
 'ACT4213_02': {0.06931123762376239},
 'ACT4253_01': {0.06435829207920793},
 'ACT4253_02': {0.06435829207920793},
 'ACT4253_03': {0.06435829207920793},
 'ACT4253_04': {0.06435829207920793},
 'ACT4311': {0.04621415841584159},
 'BIM3001': {0.04621415841584159},
 'BIM3009': {0.02641237623762376},
 'BIM3011': {0.01981178217821782},
 'BIM3019': {0.01981178217821782}

In [11]:
# 约束条件3(modified)

slack_vars = []
weighted_slack_vars = []

# 构建字典用于：课程名找活动名
module_dict = {}
# 遍历获取课程，活动数
for index, row in df_code_count.iterrows():
    module_code = row['Module Code']
    activity_counts = row['Activity Counts']
    
    # 得到每个课程的activity list
    if activity_counts > 1:
        module_list = [f"{module_code}_{i+1:02d}" for i in range(activity_counts)]
    else :
        module_list = [f"{module_code}"]
    module_dict[module_code] = module_list
    # print(f"{module_code}: {module_list}")

for course in module_dict.keys():
    if ModuleCode_Pos_Mandatory.get(module_dict[course][0]) is None:
        print(course)
        continue
     
    # 只有当Mandatory字典中存在course名时，才进行约束条件处理
    else:
        for pos in ModuleCode_Pos_Mandatory.get(module_dict[course][0]) :
                
            # 添加松弛变量
            slack_var = course_optimize.addVar(lb=0, name=f"slack_{pos}_{course}") 
            slack_vars.append(slack_var)
            
            weight = next(iter(ModuleCode_Weight_Mandatory[module_dict[course][0]]))
            weighted_slack_var = weight * slack_var
            weighted_slack_vars.append(weighted_slack_var)
            
            # course_optimize.addConstr(gp.quicksum(X[Pos.index(pos), Courses.index(activity),k] for activity in module_dict[course] for k in range(t)) + slack_var == 1, name=f"mand_constr_{pos}_{course}")
            course_optimize.addConstr(gp.quicksum(X[Pos.index(pos), Courses.index(activity), k] for activity in module_dict[course] for k in range(t)) + slack_var >= 1, name=f"mand_constr_{pos}_{course}")
            activities = []

ACT4214
BIM2006
BIO3204
BIO3207
BIO3213
BIO4201
BIO4202
BIO4204
DMS3210
IDE3002
MGT3070
PHM3005


In [12]:
#约束条件4（逻辑约束）
y = course_optimize.addVars(m, 8, vtype=GRB.BINARY, name="y") #A binary variable indicating whether a course is assigned to a specific time slot

for j in range(m):
    for k in range(t):
        course_optimize.addConstr(gp.quicksum(X[i, j, k] for i in range(n)) <= 1e6 * y[j, k])
        course_optimize.addConstr(y[j, k] <= 1e6 * gp.quicksum(X[i, j, k] for i in range(n)))

for j in range(m) :
    course_optimize.addConstr(gp.quicksum(y[j,k] for k in range(8)) == 1)

ModuleCode_Weight_Mandatory = df_Mandatory.groupby('Module Code')['Normalized Weight'].apply(set).to_dict()
ModuleCode_Weight = df_whole.groupby('Module Code')['Normalized Weight'].apply(set).to_dict()
Weights, x = gp.multidict(ModuleCode_Weight_Mandatory)

"""slack_vars_2 = []
weighted_slack_vars_2 = []
for course in module_dict.keys():
    
    # 检查course是否存在（如GE课程，存在于非TUT teample中，但不存在于ModuleCode_Pos中）
    if ModuleCode_Pos.get(module_dict[course][0]) is None:
        continue
    
    # 若存在，则添加松弛变量
    else:
        # 添加松弛变量
        slack_var_2 = course_optimize.addVar(lb=0, name=f"slack_2_{course}") 
        slack_vars_2.append(slack_var_2)
        course_optimize.addConstr(gp.quicksum(y[Courses.index(activity), k] for activity in module_dict[course] for k in range(8)) + slack_var_2 == 1)
        # course_optimize.addConstr(gp.quicksum(y[Courses.index(activity), k] for activity in module_dict[course] for k in range(8)) == 1)
        for activity in module_dict[course]:
            weight = next(iter(ModuleCode_Weight[activity]))
            weighted_slack_var_2 = weight * slack_var_2
            weighted_slack_vars_2.append(weighted_slack_var_2)
            print(f"{course}:{activity}:{Courses.index(activity)}:{weight}")   
        print("-------------------------------")"""

'slack_vars_2 = []\nweighted_slack_vars_2 = []\nfor course in module_dict.keys():\n    \n    # 检查course是否存在（如GE课程，存在于非TUT teample中，但不存在于ModuleCode_Pos中）\n    if ModuleCode_Pos.get(module_dict[course][0]) is None:\n        continue\n    \n    # 若存在，则添加松弛变量\n    else:\n        # 添加松弛变量\n        slack_var_2 = course_optimize.addVar(lb=0, name=f"slack_2_{course}") \n        slack_vars_2.append(slack_var_2)\n        course_optimize.addConstr(gp.quicksum(y[Courses.index(activity), k] for activity in module_dict[course] for k in range(8)) + slack_var_2 == 1)\n        # course_optimize.addConstr(gp.quicksum(y[Courses.index(activity), k] for activity in module_dict[course] for k in range(8)) == 1)\n        for activity in module_dict[course]:\n            weight = next(iter(ModuleCode_Weight[activity]))\n            weighted_slack_var_2 = weight * slack_var_2\n            weighted_slack_vars_2.append(weighted_slack_var_2)\n            print(f"{course}:{activity}:{Courses.index(activity)}:{weigh

In [13]:
# 限制条件5
# 读取清洗后的教授和课程数据
#
prof_activity_df = pd.read_excel('prof-activity.xlsx', sheet_name = 'Sheet1')
# print(prof_activity_df.head(10))

# # 转换为字典 {教授: [课程代码]}
# instructor_courses = prof_activity_df.groupby('Course Instructor&New/Part-Time')['Module Code'].apply(list).to_dict()
# print(instructor_courses)

# # 约束条件5: 每个教授每个时间段只能参与一个活动
# for instructor, courses in instructor_courses.items():
#     for k in range(t):
#         course_optimize.addConstr(gp.quicksum(X[i, j, k] for i in range(n) for j in courses) <= 1)

# 创建教授的课程集合字典
prof_courses = {}
for _, row in prof_activity_df.iterrows():
    prof_name = row['professor']
    course = row['Module Code']
    if prof_name not in prof_courses:
        prof_courses[prof_name] = set()
    prof_courses[prof_name].add(course)

# 打印 Courses 和 prof_courses

# print("Prof Courses:", prof_courses)

# 添加约束
# for prof_name, courses in prof_courses.items():
#     for k in range(t):
#         # 确保每个教授在每个时间段最多有一个活动安排
#         course_optimize.addConstr(
#             gp.quicksum(X[i, Courses.index(j), k] for i in range(n) for j in courses if j in Courses) <= 1,
#             name=f"prof_{prof_name}_timeslot_{k}"
#         )

# 添加约束
for prof_name, courses in prof_courses.items():
    for k in range(t):
        # 确保每个教授在每个时间段最多有一个活动安排
        course_optimize.addConstr(
            gp.quicksum(y[course_to_index[j],k] for j in courses if j in Courses) <= 1,
            name=f"prof_{prof_name}_timeslot_{k}"
        )
        
    # 确保构建的字典能够正确对应教授与活动
    for course in courses:
        if course in Courses:
            if course != index_to_course[course_to_index[course]]:
                print(f"{prof_name}:{courses}:{course}")

In [14]:
# 约束条件7

cts_ModuleCode = ['ENG1002', 'CHI1000', 'ENG2002B', 'ENG2002S', 'GFN1000', 'GFH1000', 
                  'GEB', 'GEC', 'GED', 'GEA2000']
cts_boundary = [9, 3, 5, 4, 3, 3, 4, 5, 4, 4]
cts_courses = {}
for module_code in cts_ModuleCode:
    if module_code in module_dict:
        cts_courses[module_code] = module_dict[module_code]
    
print(cts_courses)

cts_index = 0
for module_code, activities in cts_courses.items():
    activities_count = len(activities)
    print(activities_count)
    for k in range(t):
        course_optimize.addConstr(
            gp.quicksum(y[Courses.index(j), k] for j in activities if j in Courses) <= cts_boundary[cts_index],
            name=f"continuous_module_{module_code}_timeslot_{k}"
        )
    
    cts_index += 1

{'ENG1002': ['ENG1002_01', 'ENG1002_02', 'ENG1002_03', 'ENG1002_04', 'ENG1002_05', 'ENG1002_06', 'ENG1002_07', 'ENG1002_08', 'ENG1002_09', 'ENG1002_10', 'ENG1002_11', 'ENG1002_12', 'ENG1002_13', 'ENG1002_14', 'ENG1002_15', 'ENG1002_16', 'ENG1002_17', 'ENG1002_18', 'ENG1002_19', 'ENG1002_20', 'ENG1002_21', 'ENG1002_22', 'ENG1002_23', 'ENG1002_24', 'ENG1002_25', 'ENG1002_26', 'ENG1002_27', 'ENG1002_28', 'ENG1002_29', 'ENG1002_30', 'ENG1002_31', 'ENG1002_32', 'ENG1002_33', 'ENG1002_34', 'ENG1002_35', 'ENG1002_36', 'ENG1002_37', 'ENG1002_38', 'ENG1002_39', 'ENG1002_40', 'ENG1002_41', 'ENG1002_42', 'ENG1002_43', 'ENG1002_44', 'ENG1002_45', 'ENG1002_46', 'ENG1002_47', 'ENG1002_48', 'ENG1002_49', 'ENG1002_50', 'ENG1002_51', 'ENG1002_52', 'ENG1002_53', 'ENG1002_54', 'ENG1002_55', 'ENG1002_56', 'ENG1002_57', 'ENG1002_58', 'ENG1002_59', 'ENG1002_60', 'ENG1002_61'], 'CHI1000': ['CHI1000_01', 'CHI1000_02', 'CHI1000_03', 'CHI1000_04', 'CHI1000_05', 'CHI1000_06', 'CHI1000_07', 'CHI1000_08', 'CHI1000

In [15]:
# 约束条件8

k1 = (0, 1, 2, 3)
k2 = (4, 5, 6, 7)

for prof_name, courses in prof_courses.items():
    for k_interval in [k1, k2]:
        # 确保每个教授在每个时间段最多有三个活动安排
        course_optimize.addConstr(
            gp.quicksum(
                gp.quicksum(y[Courses.index(j),k] for j in courses if j in Courses)
                for k in k_interval) <= 3,
            name=f"prof_{prof_name}_ub_timeslot_{k_interval}"
        )

In [16]:
# 约束条件9（限制特定课程时间段）
Whole = {}
for pos in Pos:
    i = pos_to_index[pos]
    course_i = []
    for course in Courses:
        if pos in ModuleCode_Pos.get(course, []):
            course_i.append(course_to_index[course])
    Whole[i] = course_i

Duration_ModuleCode = df_whole.groupby('Duration')['Module Code'].apply(set).to_dict()
Duration_ModuleCode_Mandatory = df_Mandatory.groupby('Duration')['Module Code'].apply(set).to_dict()
# print(Duration_ModuleCode_Mandatory)

long_courses = [module for duration, modules in Duration_ModuleCode.items() if duration == 180 for module in modules]

median_courses_Mandatory = [module for duration, modules in Duration_ModuleCode_Mandatory.items() if duration == 120 for module in modules]
long_courses_Mandatory = [module for duration, modules in Duration_ModuleCode_Mandatory.items() if duration == 180 for module in modules]

for j in long_courses:
    print(j)

# 时长>120分钟课程限制在3,7两个时间段
k3 = (0, 1, 2, 4, 5, 6)

for i in range(n):
    for j in long_courses:
        course_optimize.addConstr(
            gp.quicksum(y[course_to_index[j], k] for k in k3) == 0, name=f"long_module_{j}_ub")

# 对于时长在90-120分钟内以及120分钟以上的专业课，不能放在下午相邻时间段（分专业考虑）
for i in range(n):
    whole_i = Whole[i]
    # 对于i专业，Day1/2下午不能连上较长以及长的专业课
    course_optimize.addConstr(
        gp.quicksum(X[i, j, 3] for j in whole_i if index_to_course[j] in long_courses_Mandatory) + gp.quicksum(X[i, j, 2] for j in whole_i if index_to_course[j] in median_courses_Mandatory) <= 1,
        name=f"pos_{i}_median&long_module_day1"
        )
    course_optimize.addConstr(
        gp.quicksum(X[i, j, 7] for j in whole_i if index_to_course[j] in long_courses_Mandatory) + gp.quicksum(X[i, j, 6] for j in whole_i if index_to_course[j] in median_courses_Mandatory) <= 1,
        )
    # print(f"{index_to_pos[i]}")

BME3200
BME4010
PHM2003
BIO2104
ACT4214
ENE3002
MIS2011
ECO4001
MKT4120
CSC4100
BME4009
MGT3650
MGT4020
MSE2023
PHY1910
TRA4030
CHM3418
DMS3003
MGT3250
CHM2118
MGT4210
MGT4030_01
MGT4030_02
ECE2810_01
MSE3023
ENB4001
PHY1920
ECE2810_02
FIN3370
CHM4328
MKT4080
PHY1002
MKT3310


In [17]:
# 小语种课程限制（需要拆分专业才能避免一个专业重复选课）

"""k6 = (0, 1, 2, 3, 6, 7)
language_course = ['SPN1002', 'FRN1002', 'JAP1002', 'KOR1002']

for i in range(n):
    whole_i = Whole[i]
    course_optimize.addConstr(
        gp.quicksum(gp.quicksum(X[i, j, k] for j in whole_i if index_to_course[j] in language_course) for k in k6) == 0,
        name=f"pos_{i}_language_ub"
    )"""

'k6 = (0, 1, 2, 3, 6, 7)\nlanguage_course = [\'SPN1002\', \'FRN1002\', \'JAP1002\', \'KOR1002\']\n\nfor i in range(n):\n    whole_i = Whole[i]\n    course_optimize.addConstr(\n        gp.quicksum(gp.quicksum(X[i, j, k] for j in whole_i if index_to_course[j] in language_course) for k in k6) == 0,\n        name=f"pos_{i}_language_ub"\n    )'

In [18]:
#教授偏好限制

# 加载文件
file_path = '更新后的总教授偏好.xlsx'
prof_preference_data = pd.read_excel(file_path)

# 定义时间段列，并映射到整数索引
time_slot_columns = {
    '0一三8:30': 0,
    '1一三10:30': 1,
    '2一三1:30': 2,
    '3一三3:30': 3,
    '4二四8:30': 4,
    '5二四10:30': 5,
    '6二四1:30': 6,
    '7二四3:30': 7
}

# 生成字典：键为Module Code，值为对应时间段索引的列表
not_prefer_timeslot = {
    row['Module Code']: [time_slot_columns[slot] for slot in time_slot_columns if row[slot] == 0]
    for _, row in prof_preference_data.iterrows()
    if row['Module Code'] in Courses
}

# 去除value为空的键值对
not_prefer_timeslot = {k: v for k, v in not_prefer_timeslot.items() if v}

# 打印字典
print(not_prefer_timeslot)

for activity, except_time_slot in not_prefer_timeslot.items():
    # 设置约束条件：sum_{k in K} sum_{i} X(i, j, k) + Slack variable == 0
    slack_var = course_optimize.addVar(lb=0, name=f"slack_{activity}") 
    slack_vars.append(slack_var)
    course_optimize.addConstr(
        gp.quicksum(y[course_to_index[activity],k] for k in except_time_slot) - slack_var == 0,
        name=f"preference_constraint_{activity}"
    )
    print(f"preference_constraint_{activity}")


{'ACT2111': [0, 4], 'ACT3011_01': [0, 1, 2, 3, 6, 7], 'ACT3011_02': [0, 1, 2, 3, 6, 7], 'ACT3011_03': [0, 1, 2, 3, 4, 5], 'ACT3121_01': [0, 1, 2, 3, 4], 'ACT3121_02': [0, 1, 2, 3, 4], 'ACT3121_03': [0, 1, 2, 3, 4], 'ACT3131_01': [0, 4], 'ACT3131_02': [0, 4], 'ACT3141_01': [4, 5, 6, 7], 'ACT3141_02': [4, 5, 6, 7], 'ACT3161_01': [3, 4, 5, 6, 7], 'ACT3161_02': [3, 4, 5, 6, 7], 'ACT3161_03': [3, 4, 5, 6, 7], 'ACT3321': [0, 1, 2, 3, 6, 7], 'ACT4111': [0, 1, 2, 3], 'ACT4213_01': [4, 5, 6, 7], 'ACT4213_02': [4, 5, 6, 7], 'ACT4214': [0, 1, 4, 5, 6, 7], 'ACT4253_01': [0, 4], 'ACT4253_02': [0, 4], 'ACT4253_03': [0, 1, 2, 3, 4, 5], 'ACT4253_04': [0, 1, 2, 3, 4, 5], 'ACT4311': [0], 'BIM2006': [0, 1, 2, 3, 4, 6, 7], 'BIM3001': [0, 1, 2, 3, 5, 6, 7], 'BIM3009': [0, 1, 2, 3, 4, 6, 7], 'BIM3011': [1, 2, 3, 4, 5, 6, 7], 'BIM3019': [0, 1, 2, 4, 5, 6, 7], 'BIO1001': [0, 1, 2, 3, 5, 6, 7], 'BIO2104': [0, 1, 2, 4, 5, 6, 7], 'BIO3207': [0, 1, 2, 3, 4, 5, 7], 'BME2001': [0, 1, 2, 4, 5, 6, 7], 'BME3200': [2, 

In [ ]:
# 目标函数直接是带权松弛变量的quicksum
course_optimize.update()
course_optimize.setObjective(gp.quicksum(slack_vars), GRB.MINIMIZE)
"""course_optimize.setParam('TuneCriterion', 0)
course_optimize.tune()"""

course_optimize.setParam('CliqueCuts', 1)
course_optimize.setParam('CoverCuts', 1)
course_optimize.setParam('CutPasses', 10)
course_optimize.setParam('MIPGap', 1e-1)
# course_optimize.setParam('SubMIPNodes', 30)
# course_optimize.setParam('NodeLimit', 30)
# course_optimize.setParam('CutOff', 360)

course_optimize.setParam('CutPasses', 2)  # 限制剪枝次数
# course_optimize.setParam('Presolve', 1)
# course_optimize.setParam('PreSparsify', 1)
# course_optimize.setParam('MIPFocus', 2)

course_optimize.optimize()
orignumvars = course_optimize.NumVars



if course_optimize.status == gp.GRB.Status.INFEASIBLE:
    print("Infeasible")
    # Compute IIS
    course_optimize.computeIIS()
    course_optimize.write("model.ilp")  # Write the IIS to a file for review

    # Print the IIS
    print("\nThe following constraints are in the IIS:")
    for constr in course_optimize.getConstrs():
        if constr.IISConstr:
            print(f"{constr.ConstrName}")

if course_optimize.status == gp.GRB.Status.INFEASIBLE:
    course_optimize.feasRelaxS(2, False, False, True)
    course_optimize.optimize()

Set parameter CliqueCuts to value 1
Set parameter CoverCuts to value 1
Set parameter CutPasses to value 10
Set parameter MIPGap to value 0.1
Set parameter CutPasses to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 87739 rows, 8148764 columns and 16963117 nonzeros
Model fingerprint: 0xbbb30d6b
Variable types: 8412 continuous, 8140352 integer (8140352 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 53793 rows and 7960540 columns (presolve time = 9s) ...
Presolve removed 53793 rows and 7960540 columns
Presolve time: 9.42s
Presolved: 33946 rows, 188224 columns, 730003 nonzeros
Variable types: 0 continuous, 188224 in

In [ ]:
slack_vars

In [ ]:
# 获取被放缩的变量名称
scaled_vars = [var.VarName for var in slack_vars if var.X == 1.0]
    
# 放缩的条件数量
print(f"放缩的条件数量为: {len(scaled_vars)}")

# 被放缩的条件的名称
print("被放缩的条件名称:")
for name in scaled_vars:
    print(name)
    
    



In [ ]:
print('\nSlack values:')
slacks = course_optimize.getVars()[orignumvars:]
for sv in slacks:
    if sv.X > 1e-9:
        print('%s = %g' % (sv.VarName, sv.X))



#检查模型是否有可行解
# if course_optimize.status == gp.GRB.Status.INFEASIBLE:
#     # 模型无可行解，进行放松
#     relax = course_optimize.feasRelaxS(0, False, False, True)
#     print(relax)
#     # 获取违反的约束条件
#     violated_constraints = relax.getConstrs()

#     # 计算违反约束的百分比
#     total_constraints = course_optimize.NumConstrs
#     violated_percentage = len(violated_constraints) / total_constraints * 100

#     print(f"违反约束的百分比: {violated_percentage}%")
# else:
#     print("无需放松")


# 获取结果
X_optimal = course_optimize.getAttr('x', X)
# 打印结果
# print("S_optimal:")
# print(S_optimal)
# print("X_optimal:")
# print(X_optimal)

In [ ]:
for constr in course_optimize.getConstrs():
    print(constr)

In [ ]:
# 将变量 X 的最优解值转换为矩阵形式
X_matrix = np.zeros((n, m, t))
for i in range(n) :
    for j in range(m):
        for k in range(t):
            X_matrix[i, j, k] = X_optimal[i, j, k]

# 输出最优解的矩阵形式

print("X_optimal:")
print(X_matrix)




In [ ]:
#课程——时间 矩阵
Y_matrix = np.zeros((m,t))
for j in range(m) :
    for k in range(t):
        sum = 0
        for i in range(n) :
            sum += X_matrix[i,j,k]
        Y_matrix[j,k] = sum

print("Y_optimal:")
print(Y_matrix)



In [ ]:
#专业——课程 矩阵
M_matrix = np.zeros((n,m))
for i in range(n) :
    for j in range(m):
        sum = 0
        for k in range(t) :
            sum += X_matrix[i,j,k]
        M_matrix[i,j] = sum

print("M_optimal:")
print(M_matrix)

In [ ]:
prof_courses

In [ ]:


# 把矩阵写入excel
# 将矩阵转换为DataFrame
df_Y_matrix = pd.DataFrame(Y_matrix)
df_M_matrix = pd.DataFrame(M_matrix)
# 将DataFrame写入Excel文件
#加入表头
new_Y_column = list(Courses)
new_M_column = list(Pos)




df_Y_matrix.insert(0, 'Course', new_Y_column)
df_Y_matrix['sum']  = df_Y_matrix.iloc[:, 1:9].sum(axis=1)


column_names = list(Courses)
df_M_matrix.columns = column_names
df_M_matrix.insert(0, 'Course', new_M_column)


df_Y_matrix.to_excel('Y_matrix(Courses-TIME).xlsx', index=False)
df_M_matrix.to_excel('M_matrix(Major-Courses).xlsx', index=False)

In [ ]:
df_Y_matrix